<a href="https://colab.research.google.com/github/fsoest/Django/blob/master/svc_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
# %%
data = pd.read_csv('https://raw.githubusercontent.com/fsoest/ml/master/avq/AVQ_data.csv')
rand = 42
# %%
total = data[['beta', 'eps_d', 'v_rad', 'n_wz', 'd0', 's_ax', 'Tu', 'alfa', 'kappa', 'v_ax', 'rillen_io']]
total.dropna(inplace=True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [0]:
total = total[total['rillen_io'] < 3]

In [24]:
total.sort_values('rillen_io', ascending=False).head()

,beta,eps_d,v_rad,n_wz,d0,s_ax,Tu,alfa,kappa,v_ax,rillen_io
153,2.0,0.2,0.2,40,50,150,1000,55,1,1.26,2.0
125,4.0,0.3,0.2,40,50,150,1000,30,10,2.67,2.0
126,4.0,0.3,0.2,40,50,150,950,30,10,2.76,2.0
183,3.5,0.2,0.2,40,50,150,1100,30,10,2.26,2.0
70,2.0,0.3,0.2,40,50,150,900,30,10,1.32,2.0


In [0]:
X = total[['beta', 'eps_d', 'v_rad', 'n_wz', 'd0', 's_ax', 'Tu', 'alfa', 'kappa', 'v_ax']]
Y = total[['rillen_io']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# %%
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, train_size=0.8, random_state=rand)

In [0]:
svc = SVC(kernel='rbf')
params = {'C': np.logspace(-5,10, 16),
  'gamma': np.logspace(-5, 5, 11),
}
grid = GridSearchCV(svc, params, n_jobs=-1)

In [29]:
grid.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05, 1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10]),
                         'gamma': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [30]:
grid.best_estimator_.score(X_test, Y_test)

0.8043478260869565

In [33]:
grid.best_estimator_

SVC(C=1000000.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [34]:
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
123,0.214348,0.021180,0.001031,0.000046,1e+06,0.001,"{'C': 1000000.0, 'gamma': 0.001}",0.819672,0.783333,0.694915,0.766667,0.052271,1
112,0.029370,0.014375,0.001143,0.000102,100000,0.001,"{'C': 100000.0, 'gamma': 0.001}",0.803279,0.783333,0.694915,0.761111,0.046941,2
133,0.415808,0.316496,0.001027,0.000042,1e+07,0.0001,"{'C': 10000000.0, 'gamma': 0.0001}",0.770492,0.783333,0.728814,0.761111,0.023159,2
91,0.012319,0.009413,0.000992,0.000016,1000,0.01,"{'C': 1000.0, 'gamma': 0.01}",0.803279,0.766667,0.677966,0.750000,0.052492,4
122,0.071466,0.022941,0.001044,0.000011,1e+06,0.0001,"{'C': 1000000.0, 'gamma': 0.0001}",0.770492,0.766667,0.711864,0.750000,0.026676,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,0.002536,0.000213,0.001056,0.000070,0.1,0.001,"{'C': 0.1, 'gamma': 0.001}",0.540984,0.550000,0.542373,0.544444,0.003969,95
47,0.004422,0.002199,0.004228,0.003494,0.1,0.01,"{'C': 0.1, 'gamma': 0.01}",0.540984,0.550000,0.542373,0.544444,0.003969,95
49,0.002691,0.000080,0.001068,0.000042,0.1,1,"{'C': 0.1, 'gamma': 1.0}",0.540984,0.550000,0.542373,0.544444,0.003969,95
42,0.002946,0.000132,0.001099,0.000013,0.01,10000,"{'C': 0.01, 'gamma': 10000.0}",0.540984,0.550000,0.542373,0.544444,0.003969,95
